# Что такое PyTorch

PyTorch — это фреймворк машинного обучения для языка Python с открытым исходным кодом, созданный на базе библиотеки Torch.

Он позволяет очень быстро и гибко описывать нейронные сети: вместо множества строк матричных умножений и ручных вычислений градиентов мы будем работать с понятными и интуитивными абстракциями.

С его помощью можно с минимальными усилиями превратить ранее написанный вручную код обучения в лаконичные и надёжные конструкции: от создания собственных датасетов до настройки функций активации и выбор оптимизатора. Это поможет вам быстрее прототипировать идеи, сосредоточиться на дизайне моделей и легко масштабировать их в дальнейшем.

PyTorch быстро стал популярным благодаря удобству разработки и гибкости. В отличие от фреймворков со статическим графом (например, TensorFlow), PyTorch использует динамический граф вычислений: граф создаётся прямо во время выполнения кода. Это делает его более гибким и упрощает отладку: вы можете изменять архитектуру сети во время работы и пользоваться привычными средствами Python (print, отладчик).

Код на PyTorch выглядит естественно для Python-разработчика и очень похож на NumPy-нотации. Кроме того, PyTorch оптимизирован под GPU: он эффективно использует видеокарты для обучения крупных моделей. Фреймворк обладает большой стандартной библиотекой слоёв, функций потерь и оптимизаторов, а также обширным сообществом исследователей и разработчиков, которые создают новые модели и обучающие материалы.

# torch.Tensor

В основе PyTorch лежит объект тензор — многомерный массив чисел, аналогичный numpy.ndarray, но с дополнительными возможностями.

Тензоры могут храниться на CPU или GPU и, при необходимости, отслеживать вычисленные градиенты.

Интерфейс PyTorch специально сделан похожим на NumPy, чтобы разработчику было удобно работать с данными.

Давайте посмотрим на несколько тензоров в действии.

In [1]:
import torch 


a = torch.tensor([1.0, 2.0, 3.0])              # 1D-тензор из списка
b = torch.zeros((2,3), dtype=torch.int64)      # матрица 2х3 из нулей(64-битные целы)
c = torch.rand(3, 3)                           # случайный тензор 3х3, равномерное распределение

print(a, a.dtype)
print(b, b.dtype)
print(c)

tensor([1., 2., 3.]) torch.float32
tensor([[0, 0, 0],
        [0, 0, 0]]) torch.int64
tensor([[0.7798, 0.4136, 0.0503],
        [0.2354, 0.7947, 0.9279],
        [0.0162, 0.9562, 0.2846]])


Тензоры поддерживают стандартные арифметические операции (сложение, умножение, матричные произведения и другие) и функциональные преобразования. Запустите код, чтобы проверить, как работают арифметические операции в PyTorch.

In [2]:
import torch


x = torch.tensor([1.0, -2.0, 3.0])
y = torch.exp(x)    # элемент-wise экспонента
z = x + y           # сложение поэлементно
print(z) 

tensor([ 3.7183, -1.8647, 23.0855])


При необходимости тензоры можно передавать на GPU:

In [3]:
if torch.cuda.is_available():
    a = a.to('cuda')  # переместить на CUDA-устройство 

Поскольку PyTorch тесно интегрирован с NumPy, тензоры можно конвертировать в numpy.ndarray и обратно — например, через torch.from_numpy или метод tensor.numpy().

# Автоматическое дифференцирование (Autograd)

PyTorch имеет встроенную систему автоматического дифференцирования (autograd), которая вычисляет градиенты. Если у тензора установлено свойство requires_grad=True, PyTorch запоминает все операции над ним и может потом вычислить градиенты по цепному правилу. Например:

In [4]:
x = torch.ones(2, 2, requires_grad=True)  # создаём тензор, указывая, что нужны градиенты
y = x * 2

z = y.mean()      # свёртка: усредняем все элементы
z.backward()      # выполняем обратный проход (backpropagation)
print(x.grad)     # смотрим градиент dz/dx

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000]])


Вот так просто мы посчитали градиент нашего тензора!

В этом примере z = (x*2).mean() = 2.0, и после z.backward() в x.grad окажутся градиенты dz/dx. Автоматическая дифференциация позволяет легко реализовать обратный проход при обучении нейросетей. Градиенты накапливаются в атрибуте .grad тензора и могут быть использованы оптимизатором для обновления параметров.

Вы увидели, как автоматическое дифференцирование тщательно отслеживает операции с тензорами и накапливает градиенты. 

Следующий шаг — организовать сами вычисления в удобные блоки. Вместо того чтобы вручную писать последовательность матричных умножений и вызов backward(), мы хотим упаковать слои сети в единый объект с встроенным хранением параметров и логикой прямого прохода. Для этого и служит класс nn.Module, который упрощает создание, хранение и обучение любой архитектуры нейронной сети.

# Модели и nn.Module

В PyTorch нейронные сети строятся из модулей, основанных на базовом классе torch.nn.Module. Модуль nn.Module — это любое преобразование данных с параметрами (например, слой сети). Все стандартные слои (линейный слой, свёрточный слой, активации и т. д.) — это подклассы nn.Module. Саму нейросеть принято реализовывать как подкласс nn.Module, где в методе __init__ задаются слои, а в forward описывается логика прямого прохода.

Подкласс nn.Module позволяет определить сразу две ключевые части любой нейросети:
-  Инициализацию, где мы создаём слои и другие компоненты.
-  Логику прямого прохода.

Рассмотрим каждый метод по отдельности.
Метод __init__ отвечает за инициализацию параметров и создание слоёв/функций активации. Когда мы наследуемся от nn.Module, первое, что нужно сделать внутри __init__, — это вызвать конструктор базового класса, чтобы внутри него успели инициализироваться внутренние механизмы PyTorch: регистрация параметров, хранение списка дочерних модулей и т. п.

In [8]:

from torch import nn


class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()  # Важно: сначала вызываем конструктор nn.Module
        # Создаём линейный слой: принимает 10 входов, выдаёт 5 выходов
        self.fc1 = nn.Linear(in_features=10, out_features=5) 
        self.relu = nn.ReLU()         # ReLU-активация 

super().__init__() запускает конструктор класса nn.Module. В результате внутри экземпляра появится структура, позволяющая хранить все слои, параметры (веса, смещения) и вложенные модули.

Без вызова super().__init__() PyTorch не увидит созданные в дальнейшем слои и не сможет корректно собрать список параметров для оптимизатора.

После инициализации базового класса можно добавить в наш класс любой набор слоёв и функций активации. Все элементы, присвоенные атрибутам экземпляра, которые сами являются подклассами nn.Module (например, nn.ReLU и т. д.), автоматически будут зарегистрированы как дочерние модули.

nn.Linear(in_features, out_features) создаёт слой, который хранит матрицу весов размера (in_features, out_features) и вектор смещений длины out_features. 

При создании по умолчанию веса и смещения инициализируются случайно — например, из нормального распределения, сконструированного отдельными правилами.

Параметры weights и bias, с которыми мы сейчас познакомимся, автоматически получают атрибут requires_grad=True, то есть по ним будут рассчитываться градиенты при расчёте обратного распространения ошибки — loss.backward().

nn.ReLU() — это слой, который не содержит обучаемых параметров, а лишь преобразует входной тензор по правилу f(z) = max(0, z). Поскольку nn.ReLU() тоже наследуется от nn.Module, PyTorch знает, что этот модуль — часть графа, и сможет правильно вставить операцию ReLU в вычислительный граф.

Когда в __init__ мы объявили все слои и функции активации, метод forward отвечает за логику прямого прохода: он получает на вход батч данных, прокатывает его через все слои, применяя необходимые преобразования, и возвращает выход. Стандартная сигнатура выглядит так:

```python

def forward(self, x):
    # x - входной тензор формы [batch_size, in_features]
    # Здесь мы пишем последовательность операций вида:
    # x = self.fc1(x)
    # x = self.relu(x)
    # x = self.fc2(x)
    # return x

```

Параметр x  должен быть тензором с той размерностью, которую ожидает первый слой. Метод forward неявно вызывается при том, что мы делаем model(input_tensor). Под капотом PyTorch перенаправляет этот вызов в forward.

<div background-color=green>
Абсолютно любой код внутри forward будет использован при построении динамического графа. Можно писать условные операторы, циклы, вызывать другие функции и т. д. Всё, что приводит к операции над тензорами с requires_grad=True, станет частью графа, по которому потом будет проходить backward().
</div>

Как работает nn.Linear:

In [6]:

import torch

from torch import nn


model = nn.Linear(in_features=10, out_features=5)
print("Весовой тензор W:", model.weight.shape)   # torch.Size([5, 10])
print("Вектор смещений b:", model.bias.shape)    # torch.Size([5])


# Проверим, что по умолчанию требуется вычисление градиентов
print(model.weight.requires_grad)
print(model.bias.requires_grad) 

Весовой тензор W: torch.Size([5, 10])
Вектор смещений b: torch.Size([5])
True
True


Проверим работу RelU:

In [9]:
import torch

from torch import nn


relu = nn.ReLU()
x = torch.tensor([[-1.0, 0.0, 2.0], [3.0, -5.0, 1.0]])
y = relu(x)
print(y)

tensor([[0., 0., 2.],
        [3., 0., 1.]])


Пошаговый пример
Допустим, есть два линейных слоя и одна активация ReLU. Логика такая:

1. На вход в forward приходит тензор x формы [batch_size, 10].
2. Сначала мы применяем линейное преобразование — есть матрица весов (10 → 5), получается тензор формы [batch_size, 5].
3. К полученному результату применяем ReLU.
4. Результат ReLU снова пропускаем через линейный слой (5 → 1), получаем [batch_size, 1].
5. Метод возвращает этот выходной тензор.

In [1]:
import torch

from torch import nn


# Предположим, что мы уже создали в __init__ следующие слои:
fc1 = nn.Linear(10, 5)     # из 10 входов в 5 нейронов
relu = nn.ReLU()
fc2 = nn.Linear(5, 1)      # из 5 входов в 1 нейрон


# А вот упрощённая функция forward, написанная вне класса:
def forward_pass(x):
    x1 = fc1(x)            # применили первый линейный слой
    x2 = relu(x1)          # применили ReLU
    x3 = fc2(x2)           # применили второй линейный слой
    return x3


# Создадим тестовый входной батч: batch_size=3, in_features=10

input_tensor = torch.randn(3, 10)
output_tensor = forward_pass(input_tensor)
print(input_tensor)
print(output_tensor)  # tensor of shape [3, 1]

tensor([[-0.7281,  0.9982,  0.7531,  0.8091,  2.0482, -0.1595, -0.8416, -0.4780,
          1.0232, -0.0787],
        [-0.9667,  0.2407, -0.3744,  0.7973,  1.7687, -0.8267,  0.1825,  0.2015,
          1.5891, -0.1798],
        [-0.0836, -0.7374, -1.4650,  1.0841,  0.0646, -0.6541,  0.8034, -0.9726,
          0.1947, -1.0811]])
tensor([[0.3407],
        [0.4364],
        [0.5301]], grad_fn=<AddmmBackward0>)


Видно, что мы неявно строим динамический граф: когда выполняются `fc1(x)`, `relu(x1)` и `fc2(x2)`, PyTorch запоминает, какие операции были применены и к каким входным тензорам. 

В дальнейшем, если мы сформируем функцию потерь от `output_tensor` и вызовем `loss.backward()`, будут вычислены градиенты по всем параметрам `fc1.weight`, `fc1.bias`, `fc2.weight`, `fc2.bias`, вы можете самостоятельно добавить в код-сниппет выводы этих параметров и посмотреть, как они будут выглядеть.

**Что должен ожидать метод forward на вход**:

- Обычно это тензор формы (batch_size, in_features) — если вы работаете с табличными данными или одним линейным слоем.

- В случае с изображениями это может быть тензор (batch_size, channels, height, width).

- Главное, чтобы форма тензора совпадала с формой, на которую настроен первый слой. Например, если первый слой — nn.Linear(10, 5), то входные данные должны иметь вторую размерность 10.

**Что должен отдавать метод forward на выход**:

- Как правило, это тензор (batch_size, out_features), где out_features — число выходных нейронов в последнем слое.
- Например, при регрессии на одно число мы получим (batch_size, 1), при классификации на 10 классов — (batch_size, 10) и т. д.
- Внутри forward мы можем применять дополнительные нелинейности, нормализации, дропауты, но результатом всегда должен быть тензор, с которым затем мы сравним выход с метками и посчитаем лосс.

In [3]:
from SimpleNN import SimpleNN
# Пример использования:
model = SimpleNN()
print(model)


# Можно сразу посмотреть, сколько параметров зарегистрировано:
total_params = sum(p.numel() for p in model.parameters())
print(f"Всего параметров: {total_params}")

SimpleNN(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=5, out_features=1, bias=True)
)
Всего параметров: 61


При вызове model(input_tensor) внутри автоматически сработает forward, и на выходе появятся предсказания.

Все параметры (fc1.weight, fc1.bias, fc2.weight, fc2.bias) автоматически попадут в model.parameters(), поэтому мы сможем передать их в оптимизатор, например:

```python
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
```

Если впоследствии мы вычислим лосс на основе предсказаний и метода loss.backward(), PyTorch аккуратно посчитает градиенты по всем весам и смещениям, а затем optimizer.step() обновит их.
Таким образом, мы подробно разобрали, как внутри nn.Module организованы ключевые методы:

    __init__ — для создания и регистрации слоёв,
    forward — для описания потока данных.

Вместе они образуют простую, но полную структуру нейросети, которую можно сразу обучать или расширять дополнительными слоями и функциональностью:

Теперь, когда наша модель определена и мы понимаем, как настроить прямой и обратный проход, возникает следующий важный вопрос: откуда будут браться данные и как они попадут внутрь нейронной сети? 

В PyTorch есть специальные инструменты — классы Dataset и DataLoader. Они позволяют:

    Не писать вручную циклы по файлам или спискам.
    Легко организовать разбиение на батчи, перемешивание и, при необходимости, загрузку данных на GPU.

# Dataset and DataLoader

Для работы с данными PyTorch предоставляет два основных примитива: Dataset и DataLoader. 

Класс torch.utils.data.Dataset — это абстрактный класс, представляющий набор данных. 

Пользовательский датасет создаётся через наследование от Dataset и переопределение методов __len__ и __getitem__.

Метод __len__ должен возвращать число образцов в датасете, а __getitem__ — один образец (обычно входные данные и метку) по индексу.

Построим простой датасет для пар (data, target).

In [2]:
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


# Создадим датасет из списков чисел
dataset = MyDataset(data=list(range(10)), targets=[2*x for x in range(10)])
print(len(dataset), dataset[0])  # 10, (0, 0)
print(dataset[1])

10 (0, 0)
(1, 2)


Мы получили нужный нам датасет, но как лучше всего его передать модели?

Класс torch.utils.data.DataLoader берёт Dataset и оборачивает его в итератор, который возвращает мини‑батчи данных.

DataLoader позволяет автоматически формировать батчи указанного размера, перемешивать данные (shuffle=True) и загружать их параллельно с помощью нескольких потоков (num_workers).

Давайте создадим свой DataLoader.

In [3]:
from torch.utils.data import DataLoader

# Создадим датасет из списков чисел
dataset = MyDataset(data=list(range(10)), targets=[2*x for x in range(10)])


dataloader = DataLoader(dataset, batch_size=3, shuffle=True, num_workers=0)
for batch_data, batch_target in dataloader:
    print(batch_data, batch_target)
    break  # посмотрим только первый батч

tensor([2, 1, 9]) tensor([ 4,  2, 18])


Разбермеся на практике:

In [4]:

import torch

from torch.utils.data import Dataset, DataLoader


# 1. Реализуйте класс ToyDataset

class ToyDataset(Dataset):
    def __init__(self, data, targets):
        # Сохраните data и targets в атрибуты
        self.data = data
        self.targets = targets


    def __len__(self):
        # Верните длину датасета
        return len(self.data)


    def __getitem__(self, idx):
        # Верните пару (вход, метка), соответствующую индексу idx
        return self.data[idx], self.targets[idx]


# 2. Подготовьте входные списки
raw_data = list(range(10)) # data = [0, 1, 2, ..., 9]
raw_targets = [2 * x for x in raw_data]  # targets = [0, 2, 4, ..., 18]


# 3. Создайте объект ToyDataset

dataset = ToyDataset(raw_data, raw_targets)


# 4. Оберните датасет в DataLoader с batch_size=4 и shuffle=True

dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


# Переберите первые два батча и выведите их на экран
for batch_idx, (batch_inputs, batch_targets) in enumerate(dataloader):
    print(f"Батч {batch_idx + 1}:")
    print("Inputs: ", batch_inputs)
    print("Targets:", batch_targets)
    if batch_idx == 1:  # остановимся после второго батча
        break

Батч 1:
Inputs:  tensor([0, 9, 2, 4])
Targets: tensor([ 0, 18,  4,  8])
Батч 2:
Inputs:  tensor([3, 6, 1, 7])
Targets: tensor([ 6, 12,  2, 14])


# Функции активации и оптимизаторы

В PyTorch уже реализовано множество функций активации и алгоритмов оптимизации. 

Функции активации (ReLU, Sigmoid, Tanh и другие) находятся в модуле torch.nn или torch.nn.functional. 
Например, nn.ReLU() применяется к выходу слоя для введения нелинейности. 

Оптимизаторы — в модуле torch.optim. Они инкапсулируют алгоритмы обновления параметров (SGD, Adam, RMSProp и т. д.). Чтобы использовать оптимизатор, нужно передать ему параметры модели и скорость обучения:

```python
import torch.optim as optim


optimizer = optim.SGD(model.parameters(), lr=0.01)     # стохастический градиентный спуск
# или
optimizer = optim.Adam(model.parameters(), lr=0.001)   # оптимизатор Адам 
```

Во время обучения на каждой итерации обычно делают так:

1. `optimizer.zero_grad()` — обнуляем накопленные градиенты у параметров.
2. Вычисляем прямой проход и функцию потерь.
3. `loss.backward()` — считаем градиенты через autograd.
4. `optimizer.step()` — обновляем параметры по посчитанным градиентам.

Это обеспечивает корректное обновление весов сети. 

Широкий выбор активаций и оптимизаторов позволяет настраивать архитектуру и обучение для разных задач.

In [2]:
import torch
from torch import nn

from SimpleNN import SimpleNN

# 2. Создаём модель и оптимизатор
model = SimpleNN(in_features=5)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

torch.manual_seed(0)
# 3. Генерируем случайные входные данные и метки
x = torch.randn(3, 5)
y = torch.randn(3, 1)


# 4. Сохраняем веса первого слоя до обучения
weights_before = model.fc1.weight.clone()


# 5. Прямой проход и лосс
outputs = model(x)
loss = criterion(outputs, y)


# 6. Шаг обучения
optimizer.zero_grad()
loss.backward()
optimizer.step()


# 7. Сохраняем веса первого слоя после обучения
weights_after = model.fc1.weight.clone()


# 8. Выводим кусочек весов до и после, а также значение loss

print("Весовой тензор fc1.weight до обучения (кусочек):")
print(weights_before[:2, :3])
print("\nВесовой тензор fc1.weight после обучения (кусочек):")
print(weights_after[:2, :3])
print(f"\nЗначение лосса: {loss.item():.4f}")

Весовой тензор fc1.weight до обучения (кусочек):
tensor([[-0.1701, -0.1830,  0.2263],
        [ 0.1084,  0.0169, -0.2946]], grad_fn=<SliceBackward0>)

Весовой тензор fc1.weight после обучения (кусочек):
tensor([[-0.1726, -0.1823,  0.2288],
        [ 0.1081,  0.0170, -0.2950]], grad_fn=<SliceBackward0>)

Значение лосса: 0.1942


Как можно видеть, тензоры весов после обучения действительно изменились

Вы рассмотрели ключевые компоненты PyTorch: 

- тензоры для хранения данных,
- Autograd для автоматического подсчёта градиентов,
- класс nn.Module для построения нейросетей,
- а также Dataset и DataLoader для организации данных.

Вы узнали, как применять функции активации и оптимизаторы для обучения моделей.

Благодаря этому PyTorch предоставляет разработчикам гибкий и интуитивно понятный инструментарий для исследований и быстрого прототипирования нейросетей.

Следующий шаг — сделать проекты более прозрачными и воспроизводимыми, даже если они растут до десятков разных запусков и конфигураций. В следующем уроке вы перейдёте от локального эксперимента к систематическому трекингу и анализу результатов: познакомитесь с фреймворком ClearML и научитесь интегрировать его в проект.